In [1]:
import numpy as np
from collections import defaultdict
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, IterableDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.tensorboard import SummaryWriter

import gymnasium as gym
import ray

/homes/crowleyd/agaid/venv_agaid/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-23 01:58:19,794	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
if not ray.is_initialized():
    ray.init()

2024-05-23 01:58:25,063	INFO worker.py:1749 -- Started a local Ray instance.


In [ ]:
with open('plot_st_matrix.py', 'r') as file:
    exec(file.read())

In [3]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from pathlib import Path

rootdir = Path('/homes/crowleyd/agaid/runs/Pendulum-v1/200k_actors__test_ray')

# data_size_to_model_map[data_size][model_type] = event_accumulator
data_size_to_model_map = dict()
for data_size_dir in rootdir.iterdir():
    data_size = int(data_size_dir.name[len('data_size_'):])
    model_type_to_event_accumulator = dict()
    for model_dir in data_size_dir.iterdir():
        model_type = model_dir.name
        event_accumulator = EventAccumulator(model_dir)
        model_type_to_event_accumulator[model_type] = event_accumulator
    data_size_to_model_map[data_size] = model_type_to_event_accumulator

event_acc = EventAccumulator(logdir)
# event_acc.Reload()
# Show all tags in the log file
print(event_acc.Tags())

# E. g. get wall clock, number of steps and value for a scalar 'Accuracy'
# w_times, step_nums, vals = zip(*event_acc.Scalars('Accuracy'))

1000
46415
100
21544
100000
2154
215
464
10000
4641
{'images': [], 'audio': [], 'histograms': [], 'scalars': [], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}


In [10]:
logdir = ('/homes/crowleyd/agaid/runs/Pendulum-v1/200k_actors__test_ray/data_size_100/embed')
ea = EventAccumulator(logdir).Reload()
# ea.Reload()
ea.Tags()

{'images': [],
 'audio': [],
 'histograms': [],
 'scalars': ['train/mean loss',
  'train/state element 0/mean loss',
  'train/state element 1/mean loss',
  'train/state element 2/mean loss',
  'train/task losses/mean loss',
  'train/task losses/task_0/state element 0',
  'train/task losses/task_0/state element 1',
  'train/task losses/task_0/state element 2',
  'train/task losses/task_1/state element 0',
  'train/task losses/task_1/state element 1',
  'train/task losses/task_1/state element 2',
  'train/task losses/task_2/state element 0',
  'train/task losses/task_2/state element 1',
  'train/task losses/task_2/state element 2',
  'train/task losses/task_3/state element 0',
  'train/task losses/task_3/state element 1',
  'train/task losses/task_3/state element 2',
  'train/task losses/task_4/state element 0',
  'train/task losses/task_4/state element 1',
  'train/task losses/task_4/state element 2',
  'train/task losses/task_5/state element 0',
  'train/task losses/task_5/state elemen

In [24]:
key = ea.scalars.Keys()[0]
print(key)
ea.scalars.Items(key)

train/mean loss


[ScalarEvent(wall_time=1714662033.6424186, step=0, value=12.817377090454102),
 ScalarEvent(wall_time=1714662065.7381232, step=1, value=7.632518291473389)]

In [ ]:
ea.scalars.FilterItems(

In [13]:
dir(ea.scalars)

['AddItem',
 'FilterItems',
 'Items',
 'Keys',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_buckets',
 '_mutex',
 'always_keep_last',
 'size']

In [94]:
x = torch.arange(10*3*1).reshape(5, 6)
t = torch.randint(0, 3, x.size()[:-1])

In [104]:
print('x', x)
print()
print('t', t)
print()
l = x[t == 0]
print('l', l)
l.mean(dim=0, dtype=float)
l.type(torch.float).mean(dim=0)
# idx = torch.tensor(idx, dtype=int).unsqueeze(0)
# torch.take_along_dim(x, idx, dim=0)

x tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29]])

t tensor([2, 1, 0, 1, 0])

l tensor([[12, 13, 14, 15, 16, 17],
        [24, 25, 26, 27, 28, 29]])


tensor([18., 19., 20., 21., 22., 23.])

In [38]:
t = torch.tensor([[10, 30, 20], [60, 40, 50]])
max_idx = torch.argmax(t)
torch.take_along_dim(t, max_idx)
sorted_idx = torch.argsort(t, dim=1)
torch.take_along_dim(t, sorted_idx, dim=1)

tensor([[10, 20, 30],
        [40, 50, 60]])

In [11]:
x = torch.rand(10, 4)
y = torch.zeros(10, 4)
print(f"error: {x - y}")
print(f"error^2: {(x - y)**2}")

F.mse_loss(x, y, reduction='none').size()

error: tensor([[0.9227, 0.1393, 0.3681, 0.6691],
        [0.6829, 0.2197, 0.8164, 0.6478],
        [0.7188, 0.5352, 0.4385, 0.5393],
        [0.1403, 0.3910, 0.8263, 0.3960],
        [0.5600, 0.0766, 0.3439, 0.3028],
        [0.7283, 0.7004, 0.3392, 0.9929],
        [0.7415, 0.8326, 0.9023, 0.8434],
        [0.6863, 0.9273, 0.9516, 0.4704],
        [0.2948, 0.5546, 0.1300, 0.7835],
        [0.5027, 0.6335, 0.4883, 0.5011]])
error^2: tensor([[0.8514, 0.0194, 0.1355, 0.4477],
        [0.4664, 0.0483, 0.6665, 0.4196],
        [0.5167, 0.2865, 0.1923, 0.2908],
        [0.0197, 0.1529, 0.6828, 0.1568],
        [0.3137, 0.0059, 0.1182, 0.0917],
        [0.5304, 0.4905, 0.1151, 0.9859],
        [0.5498, 0.6933, 0.8141, 0.7114],
        [0.4711, 0.8598, 0.9055, 0.2213],
        [0.0869, 0.3076, 0.0169, 0.6139],
        [0.2527, 0.4013, 0.2384, 0.2511]])


torch.Size([10, 4])

In [33]:
t = torch.tensor([[10, 30, 20], [60, 40, 50]])
max_idx = torch.argmax(t)
torch.take_along_dim(t, max_idx, )
sorted_idx = torch.argsort(t, dim=1)
torch.take_along_dim(t, sorted_idx, dim=1)



tensor(3)

In [22]:
t = torch.arange(10)
print(t.device)
print(t.to('cpu') is t)
print(t.device)

cpu
True
cpu


In [43]:
env = gym.make("Pendulum-v1", render_mode=None)#"human")
for e in dir(env):
    if e[0] != '_': print(e)

action_space
class_name
close
env
get_wrapper_attr
metadata
np_random
observation_space
render
render_mode
reset
reward_range
spec
step
unwrapped
wrapper_spec


In [40]:
observation, info = env.reset(seed=42)
observation

array([-0.14995256,  0.9886932 , -0.12224312], dtype=float32)

In [44]:
env = gym.make("Pendulum-v1")#, render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(1000):
    action = env.action_space.sample()  # this is where you would insert your policy
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        print(f"global_iter: {_}, terminated: {terminated}, truncated: {truncated}")
        observation, info = env.reset()

env.close()

global_iter: 199, terminated: False, truncated: True
global_iter: 399, terminated: False, truncated: True
global_iter: 599, terminated: False, truncated: True
global_iter: 799, terminated: False, truncated: True
global_iter: 999, terminated: False, truncated: True


### torch.utils.data.Dataset/DataLoader

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from dataset import CustomDataset, custom_collate_fn

# Dummy data (for demonstration purpose)
data = torch.randn(100, 2)  # 100 samples, each with 2 features

# Create an instance of your custom dataset
custom_dataset = CustomDataset(data)

# Create a DataLoader with dynamic trajectory sampling
batch_size = 10
shuffle = True
num_workers = 2
trajectory_sampling = True  # Set to True to sample trajectories, False for individual transitions

# Dynamically set trajectory sampling when creating DataLoader
dataloader = DataLoader(custom_dataset, 
                        batch_size=batch_size, 
                        shuffle=shuffle, 
                        num_workers=num_workers,
                        collate_fn=lambda batch: custom_collate_fn(batch, trajectory_sampling))

# Iterate over batches
for batch in dataloader:
    # Batch is a tuple containing input tensors
    inputs = batch
    if trajectory_sampling:
        print("Trajectory batch shape:", inputs.shape)  # shape of the batch when sampling trajectories
    else:
        print("Individual transition batch shape:", inputs.shape)  # shape of the batch when sampling individual transitions
    # Do whatever processing you need here


PicklingError: Can't pickle <function <lambda> at 0x1057f13a0>: attribute lookup <lambda> on __main__ failed

In [6]:
import sys
import random
import re
import time
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict
from importlib import  import_module, reload

# import ipdb as pdb #  for debugging
# from ipdb import set_trace
import numpy as np

from mt_model_buffer import MT_Model_Buffer

In [3]:
class Args:
    env_id = 'Pendulum-v1_old'
    buffer_name_pattern = "min_total_steps_100000__actor__[\w-]+\.pkl"
args = Args()

In [4]:
path_to_agaid = Path.cwd()
assert path_to_agaid.name == "agaid"
buffer_tasks_dir = path_to_agaid / f"offline_data/{args.env_id}"
assert buffer_tasks_dir.is_dir(), f"buffer_tasks_dir does not exist.\nbuffer_tasks_dir:\n {buffer_tasks_dir}"
# agaid / offline_data / env_id / task_val / buffers

pattern = re.compile(args.buffer_name_pattern)
task_buffer_paths = [] # One name for each task
for buffer_dir in buffer_tasks_dir.iterdir():
    buffer_paths = []
    for buffer_path in buffer_dir.iterdir():
        if pattern.fullmatch(buffer_path.name): buffer_paths.append(buffer_path)
    assert len(buffer_paths) == 1, f"buffer_name_pattern must match exactly 1 buffer file per task subfolder "\
        f"but instead matched {len(buffer_paths)}.\nbuffer_dir: {buffer_dir}\nbuffer_paths: {buffer_paths}"
    task_buffer_paths += buffer_paths
# print(f"buffer_names:", *(path.name for path in task_buffer_paths), sep='\n')

# data_sizes = np.flip(np.sort(np.geomspace(100000, 100, 10).astype(int))) # In descending order of size.
data_sizes = np.flip(np.sort(np.geomspace(5000, 100, 2).astype(int))) # In descending order of size.

# Check if CUDA is available (i.e., GPU is available)
if torch.cuda.is_available(): device = torch.device("cuda") # Use GPU
else: device = torch.device("cpu") # Use CPU

# Load buffers for all tasks.
task_buffers = []
for task_buffer_path in tqdm(task_buffer_paths[:2]):
    buffer = MT_Model_Buffer.load(task_buffer_path)
    task_buffers.append(buffer)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:03<00:00, 31.58s/it]


In [5]:
def shuffle_episodes(self):
    """
    Shuffle episodes, i.e. trajectories.
    """
    if not self._trimmed:
        self._trim_buffer()
    random.shuffle(self.tasks)
    random.shuffle(self.states)
    random.shuffle(self.actions)
    random.shuffle(self.next_states)
    random.shuffle(self.dones)

from types import MethodType

for task_buffer in task_buffers:
    task_buffer.shuffle_episodes = MethodType(shuffle_episodes, task_buffer)

In [6]:
[len(b.states) for b in task_buffers]

[1001, 1001]

In [7]:
[b.states is None for b in task_buffers]

[False, False]

In [25]:
import random
import re
import time
from pathlib import Path
from collections import defaultdict
from importlib import  import_module

# import ipdb as pdb #  for debugging
from ipdb import set_trace
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.utils.tensorboard import SummaryWriter
import tyro
from dataclasses import dataclass
from copy import deepcopy
import gymnasium as gym
import ray

from mt_model_buffer import MT_Model_Buffer
from nn.world_model import Dynamics_Model_Multihead, Dynamics_Model_Embed, Dynamics_Model_Aggregate


@dataclass
class Args:
    env_id: str = "Pendulum-v1"
    """The id of the environment"""
    exp_id: str = "200k_0"
    """The id of the experiment"""
    batch_size: int = 128
    """The number of steps per mini-batch"""
    epochs: int = 5
    """The number of epochs of training"""
    lr: float = 1e-3
    """The number of epochs of training"""
    buffer_name_pattern: str = "min_total_steps_100000__actor__[\w-]+\.pkl"
    # buffer_name_pattern: str = "min_total_steps_100000__actor__Pendulum-v1__td3_continuous_action__task_g_0.0_1__1713749400"
    """The regex pattern for the unique buffer to load per task"""
    # save_interval: int = 1000
    # """The number of epochs of training"""
    # task_distribution: int = 1000
    # """The number of epochs of training"""
    # cuda: bool = True
    # """if toggled, cuda will be enabled by default"""
    
    # num_workers: int = 1
    # """the number of ray workers"""
    # num_envs: int = 1
    # """the number of parallel game environments"""
    # buffer_size: int = int(1e6)
    # """the replay memory buffer size"""


def train_model_offline(exp_name, env_id, model_type, buffer, lr, epochs, batch_size, test_size=0.2, device=None):
    
    # Get obs_size and pred_size from env.
    env = gym.make(env_id, render_mode=None)
    env_obs_size = np.product(env.observation_space.shape)
    env_act_size = np.product(env.action_space.shape)
    assert len(env.observation_space.shape) == 1 == len(env.action_space.shape)
    obs_size = env_obs_size + env_act_size
    pred_size = env_obs_size
    

    logdir = Path('runs')
    assert logdir.is_dir()
    logdir /= env_id
    logdir /= exp_name
    logdir /= model_type
    logdir.mkdir(parents=True, exist_ok=True)
    writer = SummaryWriter(logdir)
    
    num_tasks = len(np.unique(np.concatenate(buffer.tasks)))
    if model_type == 'multihead':
        model = Dynamics_Model_Multihead(obs_size=obs_size, pred_size=pred_size, num_tasks=num_tasks)
        recurrent = False
    if model_type == 'embed':
        model = Dynamics_Model_Embed(obs_size=obs_size, pred_size=pred_size, num_tasks=num_tasks, embedding_dim=3)
        recurrent = False
    if model_type == 'aggregate':
        model = Dynamics_Model_Aggregate(obs_size=obs_size, pred_size=pred_size)
        recurrent = False
    else:
        raise RuntimeError(f"Unrecognized model_type: {model_type}")
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Split buffer into train_buffer and test_buffer.
    train_buffer, test_buffer = buffer.train_test_split(test_size=test_size)

    # Train model, write logging info.
    best_avg_loss = torch.inf
    for epoch in tqdm(range(epochs)):
        # Train model for one epoch.
        task_to_losses, avg_losses_across_tasks = train_model_offline_epoch(model=model, recurrent=recurrent, buffer=train_buffer, batch_size=batch_size, optimizer=optimizer, device=device)
        
        # Save updated model.
        if avg_losses_across_tasks.mean() < best_avg_loss:
            model.save(logdir / model_type)
        
        # Log training info.
        log_train_test_info(writer=writer, train_or_test='train', task_to_losses=task_to_losses, avg_losses_across_tasks=avg_losses_across_tasks, epoch=epoch)

        # Test model for one epoch.
        task_to_losses, avg_losses_across_tasks = train_model_offline_epoch(model=model, recurrent=recurrent, buffer=test_buffer, batch_size=batch_size, optimizer=None, device=device)
        
        # Log testing info.
        log_train_test_info(writer=writer, train_or_test='test', task_to_losses=task_to_losses, avg_losses_across_tasks=avg_losses_across_tasks, epoch=epoch)


def log_train_test_info(writer, train_or_test, task_to_losses, avg_losses_across_tasks, epoch):
    assert train_or_test in ['train', 'test']

    # Write mean loss across all tasks and all state elements.
    writer.add_scalar(f"{train_or_test}/mean loss", avg_losses_across_tasks.mean(), epoch)
    # Write mean loss across all tasks for each state element.
    for state_idx in range(len(avg_losses_across_tasks)):
        writer.add_scalar(f"{train_or_test}/state element {state_idx}/mean loss", avg_losses_across_tasks[state_idx], epoch)
    for task, task_losses in task_to_losses.items():
        # Write mean loss for this task.
        writer.add_scalar(f"{train_or_test}/task losses/mean loss", task_losses.mean(), epoch)
        # Write avg loss for this task for each state element.
        for state_idx in range(len(task_losses)):
            writer.add_scalar(f"{train_or_test}/task losses/task_{task}/state element {state_idx}", task_losses[state_idx], epoch)


def train_model_offline_epoch(model, recurrent, buffer, batch_size, optimizer=None, device=None):
    """optimizer=None deactivates training. This is used for testing."""

    if optimizer is None:
        model.eval()
    else:
        model.train()

    for batch_idx, sample_batch in enumerate(buffer.sample(get_whole_trajectories=recurrent, batch_size=batch_size)):
        for e in sample_batch:
            e.to(device)
        task_batch, state_batch, action_batch, next_state_batch, done_batch, not_padding_mask = sample_batch

        # Normalize values.
        state_batch_normalized = buffer.normalize_state(state_batch, inplace=False)
        action_batch_normalized = buffer.normalize_action(action_batch, inplace=False)
        next_state_batch_normalized = buffer.normalize_state(next_state_batch, inplace=False)

        # Calculate residual.
        delta_normalized_state_batch = next_state_batch_normalized - state_batch_normalized # The learning target.

        # Get model prediction.
        observation_batch = torch.cat((state_batch_normalized, action_batch_normalized), dim=-1)
        predicted_normalized_residual_batch = model(observation_batch, task_batch) # prediction is predicted normalized next_state residual.

        # Compute loss.
        if optimizer is not None: optimizer.zero_grad()
        # loss = F.mse_loss(input=predicted_normalized_residual_batch, target=delta_normalized_state_batch) # Doesn't account for not_padding_mask.
        losses = ((predicted_normalized_residual_batch - delta_normalized_state_batch) * not_padding_mask).pow(2) # Shape: (batch_size, <max_traj_len if recurrent,> state_size).
        loss = losses.sum() / not_padding_mask.sum()
        if optimizer is not None:
            loss.backward()
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.01) #max_norm=grad_clip=?0.01
            optimizer.step()

        with torch.no_grad():
            predicted_normalized_next_state_batch = predicted_normalized_residual_batch + state_batch_normalized
            # prediction_denormalized_next_state_batch = buffer.denormalize_state(predicted_normalized_next_state_batch)
            # normalized_mses = F.mse_loss(predicted_normalized_next_state_batch, next_state_batch_normalized, reduction='none')
            avg_losses_across_tasks = losses.sum(dim=state_batch.shape()[:-1]) / not_padding_mask.sum(dim=state_batch.shape()[:-1]) # Shape: (state_size,).
            # task_batch shape: (batch_size, <max_traj_len if recurrent,>, <possibly task_size if > 1,>).
            task_to_losses = dict()
            for task in task_batch.unique():
                task_losses = losses[task_batch == task].mean(dim=0, dtype=float) # Shape: (state_size,).
                # Averaged across batch, one element per state element per task.
                task_to_losses[task] = task_losses
            return task_to_losses, avg_losses_across_tasks

In [26]:
args = Args()

In [5]:
import torch

torch.cat([])

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [1]:
import numpy as np
import torch
from mt_model_buffer import MT_Model_Buffer

buffer = MT_Model_Buffer()
for ep in range(200):
    state = np.random.rand(5)
    for step in range(100):
        action = np.random.rand(2)
        next_state = np.random.rand(5)
        done = np.random.random() < 2/100
        buffer.push(task=ep, state=state, action=action, next_state=next_state, done=done)
        if done: break
    buffer.end_trajectory()



In [2]:
buffer.size

8900

In [36]:
sub = buffer.get_subset_buffer(min_steps=3000)

In [28]:
def train_test_split(self, test_size=None, train_size=None, shuffle=True, random_seed=None, view=True):
    """
    Following some conventions of sklearn.model_selection.train_test_split:
    https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

    test_size or train_size must be a float in [0, 1], indicating the proportion of elements in that partition.

    The split is performed on whole trajectories, irrespective of their sizes.
    TODO: make this true: The split is performed on whole trajectories to approximate the desired split on transitions.
    TODO: support option to split on transitions.
    """
    self._trim_buffer()
    num_trajectories = len(self.states)
    if shuffle: self.shuffle_episodes(seed=random_seed)

    # Validate test_size and train_size to the proportion of steps in that partition.
    if not ((test_size is None) ^ (train_size is None)): # If both or neither is None:
        raise ValueError("Exactly one of test_size and train_size must be provided.")
    if train_size is not None:
        # Infer test_size from train_size.
        assert 0 <= train_size <= 1, f"train_size must be in [0, 1]. train_size: {train_size}"
        test_size = 1 - train_size
    else:
        # Infer train_size from test_size.
        assert 0 <= test_size <= 1, f"test_size must be in [0, 1]. test_size: {test_size}"
        train_size = 1 - test_size
    # train_size & test_size are in [0, 1].

    # TODO: # Convert sizes to the number of steps, rounding towards 50-50.
    # Convert sizes to the number of trajectories.
    train_size = round(num_trajectories * train_size)
    test_size = num_trajectories - train_size

    # Perform partition.
    train_test_split_buffers = [None, None]
    for partition_idx, partition_size in enumerate([train_size, test_size]):
        # Instantiate empty buffer.
        train_test_split_buffers[partition_idx] = self.__class__()
        train_test_split_buffers[partition_idx]._trim_buffer()
        # Partition and load data into buffers.
        for traj_data_name in ['tasks', 'states', 'actions', 'next_states', 'dones']:
            traj_data = getattr(self, traj_data_name)
            # Partition data.
            traj_data_partition = traj_data[:partition_size]
            # TODO: test whether view=True is broken upstream.
            if not view: traj_data_partition = deepcopy(traj_data_partition)
            # Load data.
            setattr(train_test_split_buffers[partition_idx], traj_data_name, traj_data_partition)

        # Restore buffer to a push-ready state.
        train_test_split_buffers[partition_idx]._start_trajectory()
        push_count = sum(map(lambda traj: len(traj), train_test_split_buffers[partition_idx].states))
        train_test_split_buffers[partition_idx].size = push_count

    train_buffer, test_buffer = train_test_split_buffers
    return train_buffer, test_buffer


In [35]:
train, test = train_test_split(sub, test_size=0.2, shuffle=False)
train.size, test.size, sum(map(lambda e: len(e), train.states)), sum(map(lambda e: len(e), test.tasks))

(125, 15, 125, 15)

In [61]:
train, test = sub.train_test_split(test_size=0.2, shuffle=True)

train.size, test.size, round(test.size / (test.size + train.size), 2)

(2180, 898, 0.29)

## Debug train_model_offline.py and such

In [1]:
import random
import re
import time
from pathlib import Path
from collections import defaultdict
from importlib import  import_module

# import ipdb as pdb #  for debugging
from ipdb import set_trace
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.utils.tensorboard import SummaryWriter
import tyro
from dataclasses import dataclass
from copy import deepcopy
import gymnasium as gym
import ray

from mt_model_buffer import MT_Model_Buffer
from nn.world_model import Dynamics_Model_Multihead, Dynamics_Model_Embed, Dynamics_Model_Aggregate


@dataclass
class Args:
    env_id: str = "Pendulum-v1"
    """The id of the environment"""
    exp_id: str = "200k_0"
    """The id of the experiment"""
    batch_size: int = 128
    """The number of steps per mini-batch"""
    epochs: int = 2
    """The number of epochs of training"""
    test_size: float = 0.2
    """The proportion of data kept out for testing"""
    lr: float = 1e-3
    """The number of epochs of training"""
    buffer_name_pattern: str = "min_total_steps_100000__actor__[\w-]+\.pkl"
    # buffer_name_pattern: str = "min_total_steps_100000__actor__Pendulum-v1__td3_continuous_action__task_g_0.0_1__1713749400"
    """The regex pattern for the unique buffer to load per task"""
    # save_interval: int = 1000
    # """The number of epochs of training"""
    # task_distribution: int = 1000
    # """The number of epochs of training"""
    # cuda: bool = True
    # """if toggled, cuda will be enabled by default"""
    
    # num_workers: int = 1
    # """the number of ray workers"""
    # num_envs: int = 1
    # """the number of parallel game environments"""
    # buffer_size: int = int(1e6)
    # """the replay memory buffer size"""


def train_model_offline(exp_name, env_id, model_type, buffer, lr, epochs, batch_size, test_size=0.2, device=None):
    
    # Get obs_size and pred_size from env.
    env = gym.make(env_id, render_mode=None)
    env_obs_size = np.prod(env.observation_space.shape)
    env_act_size = np.prod(env.action_space.shape)
    assert len(env.observation_space.shape) == 1 == len(env.action_space.shape)
    obs_size = env_obs_size + env_act_size
    pred_size = env_obs_size
    

    logdir = Path('runs')
    assert logdir.is_dir()
    logdir /= env_id
    logdir /= exp_name
    logdir /= model_type
    logdir.mkdir(parents=True, exist_ok=True)
    writer = SummaryWriter(logdir)
    
    num_tasks = len(np.unique(np.concatenate(buffer.tasks)))
    if model_type == 'multihead':
        model = Dynamics_Model_Multihead(obs_size=obs_size, pred_size=pred_size, num_tasks=num_tasks)
        recurrent = False
    elif model_type == 'embed':
        model = Dynamics_Model_Embed(obs_size=obs_size, pred_size=pred_size, num_tasks=num_tasks, embedding_dim=3)
        recurrent = False
    elif model_type == 'aggregate':
        model = Dynamics_Model_Aggregate(obs_size=obs_size, pred_size=pred_size)
        recurrent = False
    else:
        raise RuntimeError(f"Unrecognized model_type: {model_type}")
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Split buffer into train_buffer and test_buffer.
    train_buffer, test_buffer = buffer.train_test_split(test_size=test_size)

    # Train model, write logging info.
    best_avg_loss = torch.inf
    for epoch in range(epochs):
        # Train model for one epoch.
        task_to_losses, avg_losses_across_tasks = train_model_offline_epoch(model=model, recurrent=recurrent, buffer=train_buffer, batch_size=batch_size, optimizer=optimizer, device=device)
        
        # Save updated model.
        if avg_losses_across_tasks.mean() < best_avg_loss:
            model.save(logdir / model_type)
        
        # Log training info.
        log_train_test_info(writer=writer, train_or_test='train', task_to_losses=task_to_losses, avg_losses_across_tasks=avg_losses_across_tasks, epoch=epoch)

        # Test model for one epoch.
        task_to_losses, avg_losses_across_tasks = train_model_offline_epoch(model=model, recurrent=recurrent, buffer=test_buffer, batch_size=batch_size, optimizer=None, device=device)
        
        # Log testing info.
        log_train_test_info(writer=writer, train_or_test='test', task_to_losses=task_to_losses, avg_losses_across_tasks=avg_losses_across_tasks, epoch=epoch)


def log_train_test_info(writer, train_or_test, task_to_losses, avg_losses_across_tasks, epoch):
    assert train_or_test in ['train', 'test']

    # Write mean loss across all tasks and all state elements.
    writer.add_scalar(f"{train_or_test}/mean loss", avg_losses_across_tasks.mean(), epoch)
    # Write mean loss across all tasks for each state element.
    for state_idx in range(len(avg_losses_across_tasks)):
        writer.add_scalar(f"{train_or_test}/state element {state_idx}/mean loss", avg_losses_across_tasks[state_idx], epoch)
    for task, task_losses in task_to_losses.items():
        # Write mean loss for this task.
        writer.add_scalar(f"{train_or_test}/task losses/mean loss", task_losses.mean(), epoch)
        # Write avg loss for this task for each state element.
        for state_idx in range(len(task_losses)):
            writer.add_scalar(f"{train_or_test}/task losses/task_{task}/state element {state_idx}", task_losses[state_idx], epoch)


def train_model_offline_epoch(model, recurrent, buffer, batch_size, optimizer=None, device=None):
    """optimizer=None deactivates training. This is used for testing."""

    if optimizer is None:
        model.eval()
    else:
        model.train()

    for batch_idx, sample_batch in enumerate(buffer.sample(get_whole_trajectories=recurrent, batch_size=batch_size)):
        for e in sample_batch:
            e.to(device)
        task_batch, state_batch, action_batch, next_state_batch, done_batch, not_padding_mask = sample_batch

        # Normalize values.
        state_batch_normalized = buffer.normalize_state(state_batch, inplace=False)
        action_batch_normalized = buffer.normalize_action(action_batch, inplace=False)
        next_state_batch_normalized = buffer.normalize_state(next_state_batch, inplace=False)

        # Calculate residual.
        delta_normalized_state_batch = next_state_batch_normalized - state_batch_normalized # The learning target.

        # Get model prediction.
        observation_batch = torch.cat((state_batch_normalized, action_batch_normalized), dim=-1)
        predicted_normalized_residual_batch = model(observation_batch, task_batch) # prediction is predicted normalized next_state residual.

        # Compute loss.
        if optimizer is not None: optimizer.zero_grad()
        # loss = F.mse_loss(input=predicted_normalized_residual_batch, target=delta_normalized_state_batch) # Doesn't account for not_padding_mask.
        losses = ((predicted_normalized_residual_batch - delta_normalized_state_batch) * not_padding_mask).pow(2) # Shape: (batch_size, <max_traj_len if recurrent,> state_size).
        loss = losses.sum() / not_padding_mask.sum()
        if optimizer is not None:
            loss.backward()
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.01) #max_norm=grad_clip=?0.01
            optimizer.step()

        with torch.no_grad():
            predicted_normalized_next_state_batch = predicted_normalized_residual_batch + state_batch_normalized
            # prediction_denormalized_next_state_batch = buffer.denormalize_state(predicted_normalized_next_state_batch)
            # normalized_mses = F.mse_loss(predicted_normalized_next_state_batch, next_state_batch_normalized, reduction='none')
            avg_losses_across_tasks = losses.sum(dim=state_batch.shape()[:-1]) / not_padding_mask.sum(dim=state_batch.shape()[:-1]) # Shape: (state_size,).
            # task_batch shape: (batch_size, <max_traj_len if recurrent,>, <possibly task_size if > 1,>).
            task_to_losses = dict()
            for task in task_batch.unique():
                task_losses = losses[task_batch == task].mean(dim=0, dtype=float) # Shape: (state_size,).
                # Averaged across batch, one element per state element per task.
                task_to_losses[task] = task_losses
            return task_to_losses, avg_losses_across_tasks

args = Args()

path_to_agaid = Path.cwd()
assert path_to_agaid.name == "agaid"
buffer_tasks_dir = path_to_agaid / f"offline_data/{args.env_id}"
assert buffer_tasks_dir.is_dir(), f"buffer_tasks_dir does not exist.\nbuffer_tasks_dir:\n {buffer_tasks_dir}"
# agaid / offline_data / env_id / task_val / buffers

pattern = re.compile(args.buffer_name_pattern)
task_buffer_paths = [] # One name for each task
for buffer_dir in buffer_tasks_dir.iterdir():
    buffer_paths = []
    for buffer_path in buffer_dir.iterdir():
        if pattern.fullmatch(buffer_path.name): buffer_paths.append(buffer_path)
    assert len(buffer_paths) == 1, f"buffer_name_pattern must match exactly 1 buffer file per task subfolder "\
        f"but instead matched {len(buffer_paths)}.\nbuffer_dir: {buffer_dir}\nbuffer_paths: {buffer_paths}"
    task_buffer_paths += buffer_paths
# print(f"buffer_names:", *(path.name for path in task_buffer_paths), sep='\n') # debug

# data_sizes = np.flip(np.sort(np.geomspace(100000, 100, 10).astype(int))) # In descending order of size.
data_sizes = np.flip(np.sort(np.geomspace(5000, 100, 2).astype(int))) # In descending order of size. #  TODO:  -------------------------->RESTORE   TODO

# Check if CUDA is available (i.e., GPU is available)
if torch.cuda.is_available(): device = torch.device("cuda") # Use GPU
else: device = torch.device("cpu") # Use CPU

# Load buffers for all tasks.
task_buffers = []
for task_buffer_path in task_buffer_paths[:2]: #  TODO:  -------------------------->RESTORE   TODO
    buffer = MT_Model_Buffer.load(task_buffer_path, reinstantiate=True)
    task_buffers.append(buffer)
# print("buffer sizes:", *(buffer.size for buffer in task_buffers), sep='\n') # debug
# print("buffer manual sizes:", *(sum(map(lambda e: len(e), buffer.states)) for buffer in task_buffers), sep='\n') # debug
# print("buffer num eps:", *(len(buffer.states) for buffer in task_buffers), sep='\n') # debug

/homes/crowleyd/agaid/venv_agaid/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-02 01:27:26,232	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
for tb in task_buffers:
    print(tb.size, len(tb.states), sum(map(lambda e: len(e), tb.states)))

100000 500 100000
100000 500 100000


In [4]:
for data_size in data_sizes:
    # Subsample and combine buffers.
    # Note: these methods may share buffer memory.
    sub_task_buffers = [task_buffer.get_subset_buffer(min_steps=data_size, no_extra=True) for task_buffer in task_buffers]
    buffer = MT_Model_Buffer.merge_buffers(sub_task_buffers)
    buffer.shuffle_episodes()

    break
    # Train a model for each model_type.
    for model_type in ['multihead', 'embed', 'aggregate']:
        print(f"Beginning to train model_type {model_type}") # debug
        train_model_offline(exp_name=f"{args.exp_id}/data_size_{data_size}", env_id=args.env_id, model_type=model_type, \
                            buffer=buffer, lr=args.lr, epochs=args.epochs, batch_size=args.batch_size, test_size=args.test_size, device=device)

In [5]:
train_buffer, test_buffer = buffer.train_test_split(test_size=0.2)

In [6]:
for b_idx, b_sample in enumerate(train_buffer.sample(get_whole_trajectories=False, batch_size=args.batch_size)):
    for e in b_sample:
        # print(e)
        e.to('cpu')
    # break

In [40]:
x = [torch.arange(np.random.randint(1, 10)*3).reshape(-1, 3).to(float) for i in range(10)]
torch.std_mean(torch.from_numpy(np.concatenate(x)), dim=0)

(tensor([6.5983, 6.5983, 6.5983], dtype=torch.float64),
 tensor([ 8.0377,  9.0377, 10.0377], dtype=torch.float64))

In [28]:
??torch.from_numpy

Docstring:
from_numpy(ndarray) -> Tensor

Creates a :class:`Tensor` from a :class:`numpy.ndarray`.

The returned tensor and :attr:`ndarray` share the same memory. Modifications to
the tensor will be reflected in the :attr:`ndarray` and vice versa. The returned
tensor is not resizable.

It currently accepts :attr:`ndarray` with dtypes of ``numpy.float64``,
``numpy.float32``, ``numpy.float16``, ``numpy.complex64``, ``numpy.complex128``,
``numpy.int64``, ``numpy.int32``, ``numpy.int16``, ``numpy.int8``, ``numpy.uint8``,
and ``bool``.

.. warning::
    Writing to a tensor created from a read-only NumPy array is not supported and will result in undefined behavior.

Example::

    >>> a = numpy.array([1, 2, 3])
    >>> t = torch.from_numpy(a)
    >>> t
    tensor([ 1,  2,  3])
    >>> t[0] = -1
    >>> a
    array([-1,  2,  3])
Type:      builtin_function_or_method